# Implementation of Byte Pair Encoding
Since we want to

### Packages

In [3]:
import matplotlib.pyplot as plt
from collections import Counter

sentence = "FloydHub is the fastest way to build, train and deploy deep learning models. Build deep learning models in the cloud. Train deep learning models."


## Add suffixes

In [66]:
import re
from collections import Counter
import pandas as pd

def calFrequency(text, char_table):
    # 1. Split the text into words and add a suffix "<w>" to each word
    words = text.split(" ")
    words_with_sfx = [word + "<w>" for word in words]

    # 2. Calculate word frequency using Counter
    word_freq = Counter(words_with_sfx)

    # 3. Calculate character frequency while treating '<w>' as a single character
    text_with_sfx = " ".join(words_with_sfx)  # Join words with suffix "<w>"

    # Create a regex pattern that treats each character in char_table as a single unit
    re_sub_list = '|'.join(re.escape(char) for char in char_table)
    print(re_sub_list)
    # Find all characters, treating '<w>' and other defined characters as units
    characters = re.findall(fr'({re_sub_list}|[^ ])', text_with_sfx)  # Use [^\s] to match non-whitespace characters
    char_freq = Counter(characters)

    # 4. Convert the word and character frequency into pandas DataFrames (tables)
    word_freq_table = pd.DataFrame(word_freq.items(), columns=['Word', 'Frequency'])
    char_freq_table = pd.DataFrame(char_freq.items(), columns=['Character', 'Frequency'])

    # 5. Sort the tables by frequency in descending order
    word_freq_table = word_freq_table.sort_values(by='Frequency', ascending=False)
    char_freq_table = char_freq_table.sort_values(by='Frequency', ascending=False)

    # 6. Return the tables
    return word_freq_table, char_freq_table

## Keep word table
until size of words table = 10

In [69]:
dict_size = 10
char_table = ['<w>']
word_freq_table, char_freq_table = calFrequency(sentence, char_table)
print(char_freq_table)
i=0

# Expanding character pairs until the desired dictionary size is reached
while len(char_freq_table) != dict_size:

    char1, char2 = char_freq_table.iloc[1:3]["Character"]
    pair = char1 + char2 if char1<char2 else char2+char1
    # print(f'Iter: {i} \n Best pair: {pair} \n Tokens: {char_freq_table} \n Numbers of tokens: {len(char_freq_table)}')

    if pair not in char_table:  # Avoid duplicates
        char_table.append(pair)  # Use add to update the set
        word_freq_table, char_freq_table = calFrequency(sentence, char_table)

    i+=1
    

<w>
   Character  Frequency
8        <w>         24
13         e         16
4          d         12
1          l         11
19         n         10
9          i          9
15         a          8
2          o          7
11         t          6
10         s          6
18         r          5
6          u          4
20         p          4
21         g          3
23         .          3
22         m          3
3          y          3
7          b          2
12         h          2
24         B          1
25         c          1
0          F          1
17         ,          1
16         w          1
14         f          1
5          H          1
26         T          1
<w>|de
<w>|de|ln


KeyboardInterrupt: 

In [41]:
print(char_freq_table)

   Character  Frequency
8        <w>         24
1          l         11
19         n         10
9          i          9
13         e          9
15         a          8
2          o          7
20        de          7
11         t          6
10         s          6
4          d          5
18         r          5
21         p          4
6          u          4
24         .          3
22         g          3
23         m          3
3          y          3
12         h          2
7          b          2
25         B          1
26         c          1
0          F          1
14         f          1
17         ,          1
16         w          1
5          H          1
27         T          1


In [72]:
import re
from collections import defaultdict

def get_stats(vocab):
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i], symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

# Initial setup
sentence = "FloydHub is the fastest way to build, train and deploy deep learning models. Build deep learning models in the cloud. Train deep learning models."
words = sentence.split()
vocab = defaultdict(int)

# Tokenize the sentence
for word in words:
    tokenized_word = ' '.join(word.lower())
    vocab[tokenized_word] += 1

num_merges = 20

for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print(f"Merge #{i+1}: {best}")

print("\nFinal vocabulary:")
for word, freq in vocab.items():
    print(f"{word}: {freq}")

Merge #1: ('d', 'e')
Merge #2: ('i', 'n')
Merge #3: ('l', 'o')
Merge #4: ('de', 'e')
Merge #5: ('dee', 'p')
Merge #6: ('l', 'e')
Merge #7: ('le', 'a')
Merge #8: ('lea', 'r')
Merge #9: ('lear', 'n')
Merge #10: ('learn', 'in')
Merge #11: ('learnin', 'g')
Merge #12: ('m', 'o')
Merge #13: ('mo', 'de')
Merge #14: ('mode', 'l')
Merge #15: ('model', 's')
Merge #16: ('lo', 'y')
Merge #17: ('t', 'h')
Merge #18: ('th', 'e')
Merge #19: ('s', 't')
Merge #20: ('b', 'u')

Final vocabulary:
f loy d h u b: 1
i s: 1
the: 2
f a st e st: 1
w a y: 1
t o: 1
bu i l d ,: 1
t r a in: 2
a n d: 1
de p loy: 1
deep: 3
learning: 3
models .: 2
bu i l d: 1
models: 1
in: 1
c lo u d .: 1
